# 分析不合规合同


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
pd.options.display.max_rows = 5


In [3]:
data_dir_name = 'data'
data_dir = Path.cwd() / data_dir_name
if not data_dir.exists():
    data_dir.mkdir()
data_dir


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data')

In [4]:
lp = '2022-03-04'
tp = '2022-03-11'
# lp: last period
lp_filename = f'non-compliant_contracts-{lp}.xlsx'
lp_sheet_name = 'Sheet1'

# tp: this period
tp_filename = f'non-compliant_contracts-{tp}.xlsx'
tp_sheet_name = 'Sheet1'

# common
key_fields = ['合同编号']


## 读取上期数据


In [5]:
lp_path = data_dir / lp_filename
lp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-04.xlsx')

In [6]:
lp_db = pd.read_excel(lp_path, lp_sheet_name)
# deduplication based on 'contract number' field
lp_db = lp_db.drop_duplicates(subset=key_fields)
lp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
329,330,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-701,wrds-2021-03-1030,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
330,331,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算


## 读取本期数据


In [7]:
tp_path = data_dir / tp_filename
tp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-11.xlsx')

In [9]:
tp_db = pd.read_excel(tp_path, tp_sheet_name)
tp_db = tp_db.drop_duplicates(subset=key_fields)
tp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
368,369,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
372,373,园区运营中心,南海意库-商业,1433221,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


## 提取增量数据

从本期数据中提取本期增量数据, 为了分析本期增量情况.

使用 `pandas` 做这种操作稍微有些复杂, 基本思路如下:

1. 使用 `left join` 模式将本期数据与上期数据进行 `merge`, 本期数据作为 `left`
2. 在使用 `merge` 函数时, 需要设置 `indicator` 参数为 `True`. 这样就可以在 `merge` 的结果表中增加一列用于表明 `merge` 的方式
3. 利用 `indicator` 来创建一个 Boolean indexing, 从而可以提取出增量数据


In [10]:
db_all = tp_db.merge(
    lp_db,
    on=key_fields,
    how='left',
    indicator=True,
    suffixes=(None, '_y')
)
incremental_flag = db_all['_merge'] == 'left_only'
incremental_db = db_all[tp_db.columns][incremental_flag]
incremental_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
8,9,产园-深圳公司,南海意库,1412215,南海意库,5栋5层-502A,nhyk-2022-03-0941,招商局蛇口工业区控股股份有限公司,深圳市华达物业管理有限公司,倒签,已审批
18,19,产园-深圳公司,光明科技园,1413273,招商局光明科技园,宿舍B7栋宿舍3层-B7-320,招光加22C006,招商局光明科技园有限公司,严珩铭,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
273,280,产园-杭州公司,上海森兰美奂创库,1427213,森兰美奂创库,森兰美奂大厦A栋B座5层-530-CK246,slmhck-2021-03-1084,上海浦隽房地产开发有限公司,吴颖晔,应结未结,合同终止日期已到仍未发起流程
286,324,园区运营中心,价值工厂,1413262,价值工厂,"价值工厂1层-机械大厅-104,价值工厂1层-机械大厅-105,价值工厂1层-机械大厅110",jzgc-2022-01-0104,招商局蛇口工业区控股股份有限公司,师傅到家（深圳）科技有限公司,应结未结,已终止未结算


## 分析本期数据

分析思路:

1. 根据 `['分公司', '项目部', '情况']` 分组, 计算数量
2. 分组小计
3. 按分组小计倒排序


In [11]:
def analyze(_df):
    tp_grp = _df.groupby(['分公司', '项目部', '情况'])
    tp_grp_unstacked = tp_grp['organ_id'].count().unstack([-1, -2])
    subtotal = '--小计--'
    tp_grp_unstacked[('', subtotal)] = \
        tp_grp_unstacked.apply(lambda s: s.sum(), axis=1)

    tp_grp_stacked = tp_grp_unstacked.sort_values(
        ('', subtotal), ascending=False).stack([-1, -2]).to_frame('数量')
    tp_grp_stacked.loc[('总计', '', '')] = tp_grp['organ_id'].count().sum()

    return tp_grp_stacked.astype('int32')


In [12]:
analyze(tp_db)


数量
分公司     项目部      情况       
产园-杭州公司 --小计--          95
        上海森兰美奂创库 倒签      9
...                    ...
产园-青岛公司 蓝湾网谷     应结未结    2
总计                     302

[47 rows x 1 columns]

## 分析增量数据


In [13]:
analyze(incremental_db)


数量
分公司     项目部      情况      
产园-杭州公司 --小计--         17
        上海森兰美奂创库 倒签     5
...                    ..
园区运营中心  价值工厂     应结未结   1
总计                     44

[19 rows x 1 columns]

## 导出下发数据


In [14]:
output_dir_name = 'output'
out_dir = Path.cwd() / output_dir_name
if not out_dir.exists():
    out_dir.mkdir()

out_filename = f'{tp}-租赁平台-合同规范性检查（下发）.xlsx'

out_path = out_dir / out_filename

with pd.ExcelWriter(out_path) as writer:
    analyze(tp_db).reset_index().to_excel(writer, sheet_name='全量统计')
    analyze(incremental_db).reset_index().to_excel(writer, sheet_name='增量统计')
    tp_db.reset_index().to_excel(writer, sheet_name='全量数据')
    incremental_db.reset_index().to_excel(writer, sheet_name='增量数据')
